In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

import os
import random
from pathlib import Path

from sentence_transformers import SentenceTransformer

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
# model = SentenceTransformer("all-MiniLM-L6-v2").to(device).eval()
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2").to(device).eval()
# model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2").to(device).eval()

# Dataframe

In [8]:
dic = {'cat': [], 'path': [], 'text': []}
for e in os.walk('data'):
    
    # dirs = e[1]
    files = e[2]
    if files != []:

        for file in files:

            root = e[0]

            dic['cat'].append(root.split('/')[1])
            dic['path'].append(Path(root, file))
            dic['text'].append(Path(root, file).read_text(encoding='utf-8'))

In [248]:
df = pd.DataFrame(dic)
df['num_sentences'] = df['text'].apply(lambda x: len(x.split('\n')))
with pd.option_context('display.max_colwidth', None):
  display(df)

cat                                                          path  \
0       اجتماعی            data/اجتماعی/نظامی/بدون عنوان/دفاع مردمی/98692.txt   
1       اجتماعی           data/اجتماعی/نظامی/بدون عنوان/دفاع مردمی/102198.txt   
2       اجتماعی            data/اجتماعی/نظامی/بدون عنوان/دفاع مردمی/95609.txt   
3       اجتماعی           data/اجتماعی/نظامی/بدون عنوان/دفاع مردمی/100964.txt   
4       اجتماعی           data/اجتماعی/نظامی/بدون عنوان/دفاع مردمی/101571.txt   
...         ...                                                           ...   
17594  اطلاعاتی  data/اطلاعاتی/امنیتی/مبارزه با جنایات سازمان یافته/93104.txt   
17595  اطلاعاتی  data/اطلاعاتی/امنیتی/مبارزه با جنایات سازمان یافته/97911.txt   
17596  اطلاعاتی  data/اطلاعاتی/امنیتی/مبارزه با جنایات سازمان یافته/98286.txt   
17597  اطلاعاتی  data/اطلاعاتی/امنیتی/مبارزه با جنایات سازمان یافته/99755.txt   
17598  اطلاعاتی  data/اطلاعاتی/امنیتی/مبارزه با جنایات سازمان یافته/97999.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
df.value_counts('cat', ascending=True)

cat
اجتماعی      859
اقتصادی     1229
سیاسی       6526
اطلاعاتی    8985
Name: count, dtype: int64

# Latent Space Approach
- Zero-shot classification
- No training required
- Just need a pre-trained model

### Query

In [240]:
idx = random.randint(0, len(df))
query = model.encode(df['text'][idx])


query_text = df['text'].iloc[idx]
query_text = pd.DataFrame(pd.Series(query_text), columns=['Query'])
with pd.option_context('display.max_colwidth', None):
  display(query_text)
df['cat'].iloc[idx]

,Query
0,"ادعاي الشرق الاوسط:ايران خواهان تشكل هاي انقلابي آماده براي دفاع از طرح هاي شيعه شد\nتهران - روزنامه فرامنطقه اي ""الشرق الاوسط"" روز يكشنبه گزارش داد: يك \nمقام نزديك به (آيت الله ) ""علي خامنه اي "" گفت كه آنان خواهان نيروهاي \nانقلابي آماده براي دفاع از طرح هاي شيعه هستند \nاين گزارش افزود: دايره جنك در سوريه اختلاف بين مراجع شيعه در عراق و\nايران را به ويژه درباره اعزام مبارزان به سوريه و جنگيدن در كنار\nنيروهاي وابسته به ""بشار اسد"" را گسترش داده است \nاز زمان تهاجم به عراق به رهبري ايالات متحده آمريكا در سال 2003 و\nبراندازي ""صدام حسين "" ديكتاتور عراق - رقابت بين مراجع شيعه در نجف و قم \nبراي رهبري شيعيان افزايش يافته است \nاين گزارش افزود: در حالي كه مراجع ديني در قم داوطلبان را براي جنك و\nمبارزه در كنار نيروهاي اسد در سوريه ترغيب مي كنند- مراجع شيعه در\nنجف و در راس آنها آيت الله ""علي سيستاني "" با اعزام داوطلبان شيعه به \nسوريه براي جنگيدن مخالفت كرده و درگيري ها در اين كشور را سياسي مي \nدانند- نه مذهبي \nبا وجود آنكه آيت الله سيستاني نسبت به اعزام داوطلب به سوريه مخالف \nاست - برخي از احزاب و شبه نظاميان شيعه عراق كه به آيت الله علي \nخامنه اي وفادار هستند- به دعوت اخير وي پاسخ داده و نيروهاي خود را\nبراي جنك به سوريه اعزام كرده اند \nدر همين رابطه يكي از روحانيون شيعه كه دفتر يكي از چهار مرجع بزرك در\nشهر نجف را اداره مي كند در اين باره گفت : افراديكه براي جنك به \nسوريه رفته اند- از دستورات مراجع سرپيچي كرده اند \nروحاني شيعه نزديك به مرجعيت نجف مي گويد كه اختلاف بين مراجع قم و نجف \nاز گذشته وجود داشته - اما اين اختلاف اكنون بر سر مواضع عراق در باره \nبحران سوريه اتفاق افتاده است \nروحاني مذكور كه نام وي در اين گزارش ذكر نشده - افزود: اگر دو مرجع \nشيعه متحد هم بودند- باز ما با موضع دولت عراق كه از نظام سوريه \nحمايت مي كند- روبرو هستيم \nاين گزارش يادآوري كرد كه دولت عراق اعلام كرده در قبال درگيري ها در\nسوريه بي طرف است - اما ورود مبارزان شيعه عراقي به سوريه اين مواضع \nرا نقض مي كند \nبر پايه اين گزارش - روحانيون بزرك و سياستمداران مي گويند كه سوريه \nبراي (آيت الله ) خامنه اي و پيروانش در عراق و ايران حلقه مهمي از هلال \nشيعه به حساب مي آيد كه از تهران تا بيروت و بغداد و دمشق مي گذرد \nبنابراين گزارش - آيت الله ""كاظم الحائري "" يكي از طرفداران جنك در سوريه \nكه در تهران زندگي مي كند و رهبر ديني ""مقتدا صدر"" رييس ""جناح صدر"" مي \nباشد - در اين باره گفت كه اين جنك يك واجب و دفاع از اسلام است \nبر اساس اظهارات منابعي در شبه نظاميان عراقي - هر هفته 50 مبارز\nشيعه عراقي براي كمك به نيروهاي اسد و يا دفاع از زيارتگاه هاي مقدس \nشيعيان مانند مقام حضرت ""زينب "" (س ) در اطراف دمشق وارد سوريه مي \nشوند \nالشرق الاوسط افزود: ""علي "" عضو سابق ""ارتش المهدي "" وابسته به جناح\nمقتدا صدر در حالي كه خود را براي سفر به سوريه آماده مي كند- در اين \nباره گفت : من دستورات مراجع را اطاعت مي كنم رهبر معنوي من گفته كه \nجنك در سوريه يك واجب شرعي است \nمن به گفته ديگران اهميت نمي دهم و هيچ كسي هم نمي تواند مانع من شود \nمن از دينم و دختر امامم ( اشاره به مقام حضرت زينب ) دفاع مي كنم \nاما مدير دفتر مرجع ديني در نجف معتقد است كه ايران با استفاده از\nمساله حمايت از زيارتگاه مقدس - شيعيان شيعه را بسوي جنك مي كشاند \nاين گزارش افزود: از زمان سقوط نظام صدام حسين - ايران نفوذش را در\nاين كشور گسترش داد و تلاش كرد جاي پايي به خصوص در نجف بيابد \nبه گفته مقامات محلي - در همين راستا روحانيون بزرك ايران دفاتري در نجف \nبا عناوين نهادهاي غير دولتي و خيريه و فرهنگي تاسيس كردند كه همه آنها از\nسوي مراجع ايران و يا سفارت اين كشور در بغداد تامين مالي مي شوند \nاين گزارش اضافه كرد: پرچم ايران بر فراز ساختماني دو طبقه در منطقه \nمرفه نشين نجف برافراشته شده كه مقر موسسه امام خميني (ره ) است اين \nموسسه يكي از چندين موسسه اي است كه در عراق در زمينه هاي اجتماعي \nفعاليت مي كند و تمركز آنان روي جوانان و كمك به ازدواج و ارايه كمك \nهاي سازمان يافته به زنان بيوه و ايتام و طلبه هاي ديني است \nبه گفته روحانيون مطلع برخي از اين موسسه ها به روحانيون جوان براي \nسفرهاي زيارتي به ايران و از جمله ديدار با دفتر (آيت الله ) خامنه اي -\nخدمات رايگان ارايه مي كند \n"


'اجتماعی'

### Search

In [245]:
categories = list(set(df['cat']))
embeddings = model.encode(categories)
similarities = model.similarity(query, embeddings)
values, indices = similarities.sort(descending=True)
similarities

tensor([[0.3077, 0.1667, 0.3212, 0.2448]])

In [247]:
prediction = categories[indices[0][0]]
prediction

'سیاسی'